# eQTL boxplot

This is script ported from python to fix unknown plotting error.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(ggpubr)
})

## Functions

In [ ]:
feature = "junctions"

### Cached functions

In [ ]:
get_eqtl_df <- function(){
    eGenes_file = paste0('/ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/',
                        feature, '/expression_gct/prepare_expression/fastqtl_permutation/',
                        '_m/Brainseq_LIBD.genes.txt.gz')
    eGenes = data.table::fread(eGenes_file) %>% 
        select(gene_id, variant_id, maf, slope, slope_se, pval_nominal, qval) %>%
        arrange(qval)
    return(eGenes)
}
memEQTL <- memoise::memoise(get_eqtl_df)

get_residualized_df <- function(){
    expr_file = paste0("/ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/",
                      feature,"/expression_gct/covariates/residualized_expression/_m/",
                      feature, "_residualized_expression.csv")
    return(data.table::fread(expr_file) %>% column_to_rownames("gene_id"))
}
memRES <- memoise::memoise(get_residualized_df)

get_genotypes <- function(){
    traw_file = paste0("/ceph/projects/brainseq/genotype/download/topmed/convert2plink/",
                      "filter_maf_01/a_transpose/_m/LIBD_Brain_TopMed.traw")
    traw = data.table::fread(traw_file) %>% rename_with(~ gsub('\\_.*', '', .x))
    return(traw)
}
memSNPs <- memoise::memoise(get_genotypes)

### Simple functions

In [ ]:
feature_map <- function(feature){
    return(list("genes"="Gene", "transcripts"= "Transcript", 
                "exons"= "Exon", "junctions"= "Junction")[[feature]])
}

get_geno_annot <- function(){
    return(memSNPs() %>% select(CHR, SNP, POS, COUNTED, ALT))
}

get_snps_df <- function(){
    return(memSNPs() %>% select("SNP", starts_with("Br")))
}

letter_snp <- function(number, a0, a1){
    if(is.na(number)){ return(NA) }
    if( length(a0) == 1 & length(a1) == 1){
        seps = ""; collapse=""
    } else {
        seps = " "; collapse=NULL
    }
    return(paste(paste0(rep(a0, number), collapse = collapse), 
                 paste0(rep(a1, (2-number)), collapse = collapse), sep=seps))
}

get_snp_df <- function(variant_id, gene_id){
    zz = get_geno_annot() %>% filter(SNP == variant_id)
    xx = get_snps_df() %>% filter(SNP == variant_id) %>% 
        column_to_rownames("SNP") %>% t %>% as.data.frame %>% 
        rownames_to_column("BrNum") %>% mutate(COUNTED=zz$COUNTED, ALT=zz$ALT) %>% 
        rename("SNP"=all_of(variant_id))
    yy = memRES()[gene_id, ] %>% t %>% as.data.frame %>% 
        rownames_to_column("BrNum")
    ## Annotated SNPs
    letters = c()
    for(ii in seq_along(xx$COUNTED)){
        a0 = xx$COUNTED[ii]; a1 = xx$ALT[ii]; number = xx$SNP[ii]
        letters <- append(letters, letter_snp(number, a0, a1))
    }
    xx = xx %>% mutate(LETTER=letters, ID=paste(SNP, LETTER, sep="\n"))
    df = inner_join(xx, yy, by="BrNum") %>% mutate_if(is.character, as.factor)
    return(df)
}
memDF <- memoise::memoise(get_snp_df)

save_ggplots <- function(fn, p, w, h){
    for(ext in c('.pdf', '.png', '.svg')){
        ggsave(paste0(fn, ext), plot=p, width=w, height=h)
    }
}

get_biomart_df <- function(){
    biomart = data.table::fread("../_h/biomart.csv")
}
memMART <- memoise::memoise(get_biomart_df)

get_gene_symbol <- function(gene_id){
    ensemblID = gsub("\\..*", "", gene_id)
    geneid = memMART() %>% filter(ensembl_gene_id == gsub("\\..*", "", gene_id))
    if(dim(geneid)[1] == 0){
        return("")
    } else {
        return(geneid$external_gene_name)
    }
}

plot_simple_eqtl <- function(fn, gene_id, variant_id, eqtl_annot, prefix){
    bxp = memDF(variant_id, gene_id) %>%
        ggboxplot(x="ID", y=gene_id, fill="red", add="jitter", xlab="",
                  ylab="Residualized Expression", outlier.shape=NA,
                  add.params=list(alpha=0.5), alpha=0.4,
                  ggtheme=theme_pubr(base_size=20, border=TRUE)) +
        font("xy.title", face="bold") + 
        ggtitle(paste(prefix, gene_id, eqtl_annot, sep='\n')) +
        theme(plot.title = element_text(hjust = 0.5, face="bold"))
    print(bxp)
    save_ggplots(fn, bxp, 7, 7)
}

### GWAS plots

In [ ]:
get_gwas_snps <- function(){
    gwas_snp_file = paste0('../../summary_table/_m/Brainseq_LIBD_caudate',
                           '_4features_PGC2.signifpairs.txt.gz')
    gwas_df = data.table::fread(gwas_snp_file) %>% filter(Type == feature_map(feature)) %>%
        select(c("variant_id", "gene_id", "rsid", "hg38chrc", "OR", "SE", "P", "A1", 
                 "A2", "pval_nominal", "pgc2_a1_same_as_our_counted", "is_index_snp")) %>% 
        distinct() %>% arrange(P)
    return(gwas_df)
}
memGWAS <- memoise::memoise(get_gwas_snps)

get_gwas_snp <- function(variant){
    return(memGWAS() %>% filter(variant_id == variant))
}

get_risk_allele <- function(variant){
    gwas_snp = get_gwas_snp(variant)
    if(gwas_snp$OR > 1){
        ra = gwas_snp$A1
    }else{
        ra = gwas_snp$A2
    }
    return(ra)
}

get_eqtl_gwas_df <- function(){
    return(memEQTL() %>% inner_join(memGWAS(), by="variant_id"))
}

get_gwas_ordered_snp_df <- function(variant_id, gene_id, pgc2_a1_same_as_our_counted, OR){
    df = memDF(variant_id, gene_id)
    if(!pgc2_a1_same_as_our_counted){
        if(OR < 1){ df = df %>% mutate(SNP = 2-SNP, ID=paste(SNP, LETTER, sep="\n")) }
    } else {
        if(OR > 1){ df = df %>% mutate(SNP = 2-SNP, ID=paste(SNP, LETTER, sep="\n")) }
    }
    return(df)
}

plot_gwas_eqtl <- function(fn, gene_id, variant_id, eqtl_annot, 
                           pgc2_a1_same_as_our_counted, OR, title){
    bxp = get_gwas_ordered_snp_df(variant_id, gene_id, 
                                  pgc2_a1_same_as_our_counted, OR) %>%
        mutate_if(is.character, as.factor) %>%
        ggboxplot(x="ID", y=gene_id, fill="red", add="jitter", xlab="",
                  ylab="Residualized Expression", outlier.shape=NA,
                  add.params=list(alpha=0.5), alpha=0.4,
                  ggtheme=theme_pubr(base_size=20, border=TRUE)) +
        font("xy.title", face="bold") + ggtitle(title) +
        theme(plot.title = element_text(hjust = 0.5, face="bold"))
    print(bxp)
    save_ggplots(fn, bxp, 7, 7)
}

## Plot eQTL

In [ ]:
get_drd2_junction_annotation <- function(junction_id){
    return(list(
        'chr11:113424683-113474229(-)'= "DRD2 junction 1L-2",
        "chr11:113424683-113475075(-)"= "DRD2 junction 1-2",
        "chr11:113418137-113424366(-)"= "DRD2 junction 2-3",
        "chr11:113417000-113418026(-)"= "DRD2 junction 3-4",
        "chr11:113415612-113416862(-)"= "DRD2 junction 4-5",
        "chr11:113414462-113415420(-)"= "DRD2 junction 5-6",
        "chr11:113412884-113415420(-)"= "DRD2 junction 5-7",
        "chr11:113412884-113414374(-)"= "DRD2 junction 6-7",
        "chr11:113410921-113412555(-)"= "DRD2 junction 7-8")[[junction_id]])
}

get_drd2_junctions <- function(){
    cmd = paste0("cat <(head -1 /ceph/projects/v4_phase3_paper/analysis/differential_expression/_m/junctions/diffExpr_szVctl_full.txt)",
                 " <(grep -i drd2 /ceph/projects/v4_phase3_paper/analysis/differential_expression/_m/junctions/diffExpr_szVctl_full.txt)")
    return(data.table::fread(cmd=cmd) %>% rename("Feature"="V1"))
}

get_drd2 <- function(){
    drdj = get_drd2_junctions() %>% filter(str_detect(gencodeTx, "ENST00000362072.7|ENST00000346454.7"))
    return(memEQTL() %>% filter(gene_id %in% drdj$Feature))
}

### DRD2 plot

In [ ]:
drd2_df = get_drd2()
drd2_df

In [ ]:
for(x in seq_along(drd2_df$gene_id)){
    anno = get_drd2_junction_annotation(drd2_df$gene_id[x])
    en = gsub("-", "_", gsub(" ", "_", anno))
    fn = paste("drd2_eqtl", en, sep="_")
    eqtl_annot = paste("eQTL q-value:", signif(drd2_df$qval[x], 2))
    prefix = anno
    plot_simple_eqtl(fn, drd2_df$gene_id[x], drd2_df$variant_id[x], eqtl_annot, prefix)
    #print(prefix)
}

### GWAS association

In [ ]:
eqtl_gwas_df = memGWAS() %>% filter(gene_id %in% drd2_df$gene_id) %>%
    group_by(gene_id, is_index_snp) %>% slice(1)
eqtl_gwas_df

In [ ]:
for(num in seq_along(eqtl_gwas_df$variant_id)){
    anno = get_drd2_junction_annotation(eqtl_gwas_df$gene_id[num])
    en = gsub("-", "_", gsub(" ", "_", anno))
    variant_id = eqtl_gwas_df$variant_id[num]
    gene_id = eqtl_gwas_df$gene_id[num]
    pgc2_a1_same_as_our_counted = eqtl_gwas_df$pgc2_a1_same_as_our_counted[num]
    OR = eqtl_gwas_df$OR[num]
    eqtl_annot = paste("eQTL nominal p-value:", signif(eqtl_gwas_df$pval_nominal[num], 2))
    gwas_annot = paste("SZ GWAS pvalue:", signif(eqtl_gwas_df$P[num], 2))
    risk_annot = paste("SZ risk allele:", get_risk_allele(eqtl_gwas_df$variant_id[num]))
    title = paste(anno, gene_id, eqtl_annot, gwas_annot, risk_annot, sep='\n')
    if(eqtl_gwas_df$is_index_snp[num]){
        fn = paste("drd2_eqtl_in_gwas_significant_index_snp", en, sep="_")
    } else {
        fn = paste("drd2_eqtl_in_gwas_significant_snp", en, sep="_")
    } 
    plot_gwas_eqtl(fn, gene_id, variant_id, eqtl_annot, 
                   pgc2_a1_same_as_our_counted, OR, title)
    print(title)
}

## Session Info

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()